## Keyvi index
Keyvi is a KV store.

In comparison to other KV stores as Redis its underlining technology is not Hash function, but Finite State Trancducers (FTS).

There are two structures Index and Dictionary.
Both share the same Data format as stored on a disk.

The difference is the Dictionary is static and the Index is dynamic i.e. UPDATABLE.

Another quirk is there could be only ONE Writer but MANY Readers.

Below is example how Index work.


In [3]:
import keyvi.index

First lets create Read-Write object.

In [36]:
kv = keyvi.index.Index('test')

.... then use .Set() to create KV pair

In [ ]:
kv.Set('abc', 'abc')

.. then some more

In [ ]:
kv.Set('abcd', str(555))

In [8]:
kv.Set('abxy', '23.67')

In [58]:
kv.Set('brum','789')

In [9]:
kv.Set('abxyz', f'{int(55.12345678*100)}')

We use .Get() which returns matching object

In [38]:
match = kv.Get('abc')

we can get back the value if such exists from this object

In [39]:
match.GetValue()

'abc'

In [41]:
kv.Get('abcd').GetValue()

555

In [43]:
kv.Get('abxyz').GetValue()/100

55.120

We can also can search for a key/keys even if its not exact match by specifing Levenstain distance.

**.GetFuzzy(key, distance, len-of-exact-prefix)**

In [61]:
match2 = kv.GetFuzzy('abcd',1,1)

In [62]:
[ m.GetValue() for m in match2]

['abc', 555]

You can also open the Index as read-only as I mentioned in the begining

In [53]:
ro = keyvi.index.ReadOnlyIndex('test')

We pass string as value to .Set(), but get back the correct type ..

In [55]:
ro.Get('abxy').GetValue()

23.670

In [56]:
type(ro.Get('abaxy').GetValue())

float

.. the same for integer

In [59]:
ro.Get('brum').GetValue()

789

In [60]:
type(ro.Get('brum').GetValue())

int

Because the KV file is Memory mapped reading is Faaast... and as a bonus because of the underlying technology FST everything is compressed on the fly ... win/win 

In [63]:
%timeit ro.Get('brum').GetValue()

591 ns ± 3.12 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Storing sequences

Now I will show you how to save sequences i.e. list of integers as key, value or both. 

By default Keyvi does not support that but we can convert the list to string beforehand and from string to list on the way out.

In [70]:
import struct 

# list of numbers =to=> string
def nums2str(nums, itype='H'): #B:int8,H:int16,I:int32,Q:int64
    return struct.pack(f">{len(nums)}{itype}", *nums)

#string =2=> tuple of numbers, use list()
def str2nums(b, itype='H'):
    size = 1 if itype == 'B' else ('HIQ'.index(itype) + 1) * 2
    return struct.unpack(f">{len(b)//size}{itype}", b)# bytes(b,'utf-8'))	

In [71]:
nums2str([1,2,3,4])

b'\x00\x01\x00\x02\x00\x03\x00\x04'

In [72]:
str2nums(nums2str([1,2,3,4]))

(1, 2, 3, 4)

In [73]:
kv.Set(nums2str([1,2,3,4]), 'a list key')

In [74]:
kv.Get(nums2str([1,2,3,4])).GetValue()

'a list key'

In [75]:
kv.Set(nums2str([5,6,7]), nums2str([7,8,9]))

In [77]:
val = kv.Get(nums2str([5,6,7])).GetValue()
val

'\x00\x07\x00\x08\x00\t'

In [79]:
str2nums(bytes(val,'utf-8'))

(7, 8, 9)

> I just decided to try and seems keyvi support lists you just need to pass them as string ;)

In [80]:
kv.Set('str-list','[1,2,3]')

In [83]:
kv.Get('str-list').GetValue()

[1, 2, 3]

In [84]:
type(kv.Get('str-list').GetValue())

list

In [85]:
kv.Set('[1,2]','kwy-str-list')

In [87]:
kv.Get('[1,2]').GetValue()

'kwy-str-list'